Подключим традиционные библиотеки tensorflow, numpy. Далее мы будем использовать готовые сети из библиотеки tensornets и будем визуализировать их с помощью opencv (cv2). OpenCV обязателен для работы tensornets, так как через него оно как минимум делает преобразование изображений.

In [1]:
import tensorflow as tf
import tensornets as nets
import cv2
import numpy as np
import time


Мы подключили библиотеку tensornets, где есть основные архитектуры сетей и наборы обученных коэффициентов. Поэтому двумя строчками мы можем загрузить готовую модель YOLOv3.

In [2]:
inputs = tf.placeholder(tf.float32, [None, 416, 416, 3]) 
model = nets.YOLOv3COCO(inputs, nets.Darknet19)

W1108 18:05:01.981753  7936 deprecation_wrapper.py:119] From D:\Anaconda3\envs\Neural\lib\site-packages\tensornets\utils.py:238: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W1108 18:05:02.289569  7936 deprecation_wrapper.py:119] From D:\Anaconda3\envs\Neural\lib\site-packages\tensornets\utils.py:283: The name tf.get_variable_scope is deprecated. Please use tf.compat.v1.get_variable_scope instead.

W1108 18:05:05.806452  7936 deprecation_wrapper.py:119] From D:\Anaconda3\envs\Neural\lib\site-packages\tensornets\utils.py:246: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1108 18:05:05.822074  7936 deprecation_wrapper.py:119] From D:\Anaconda3\envs\Neural\lib\site-packages\tensornets\utils.py:125: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.



YOLO будет выдавать нам индекс класса, а не имя класса объекта, который она обнаружила. Поэтому нам надо найти таблицу соответствия индекса и класса, чтобы понимать что за объект сеть обнаружила. Это не так просто и хоть в интернете такие таблицы есть, но они разные, так как база COCO развивалась и выходили новые версии. Какую из них использовать?
Оказывается, что если порыться в пакете tensornets, то в dataset папке есть перечень классов COCO. Нехитрым дедуктивным методом через dir() и посмотрев код пакета, можно загрузить перечень классов. В описании tensornets я этой информации не нашёл.

In [3]:
import tensornets.datasets.coco as coco
print("There are {} classes in coco.classnames.".format(len(coco.classnames)))

There are 80 classes in coco.classnames.


Теперь научимся загружать картинки. Есть много способов, но нам нужна совместимость по типам с tensornets и чтобы не тратить много времени на согласование этих типов возьмём из примеров проекта tensornets на гитхабе их способ загрузки картинки. Проверим, что получилось нужное разрешение, которое должно быть 416х416х3, так как такие у нас входы сети выше.

In [4]:
imge = np.array(nets.utils.load_img('YOLO_test.jpg', target_size=416, crop_size=416))
print(imge.shape)

(1, 416, 416, 3)


Нарисуем картинку с помощью opencv. Надо будет нажать любую клавишу, чтобы программа продолжилась. Это связано с особенностями opencv, когда картинка не отрисовывается, пока мы не ставим остановку средствами opencv. Для остановки выбрано ожидание нажатия клавиши. Также из формата дробных чисел нам нужно перейти в низкоуровневый формат восьмибитного представления цвета, да еще и поменять местами красный и синий каналы, так как opencv в отличие от остальных использует BGR формат.

In [5]:
img_to_show = cv2.cvtColor(np.uint8(imge[0]), cv2.COLOR_RGB2BGR)
cv2.imshow("image", img_to_show)
cv2.waitKey(0)
cv2.destroyAllWindows()

Итак, мы видим что собрались распознавать. У нас есть картинка для визуализации, а также картинка для передачи на поиск объектов. Сделаем его, как это сделано в примере в readme.md в репозитории tensornets. В итоге мы получим рамки вокруг объектов в виде массива. Номер массива соответствует классу.

In [6]:
with tf.Session() as sess:
    sess.run(model.pretrained())
    imge = model.preprocess(imge)  # equivalent to img = nets.preprocess(model, img)
    preds = sess.run(model.preds, {inputs: imge})
    boxes = np.array(model.get_boxes(preds, imge.shape[1:3]))
    print(boxes)

W1108 18:06:07.650128  7936 deprecation_wrapper.py:119] From D:\Anaconda3\envs\Neural\lib\site-packages\tensornets\utils.py:130: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.



[array([[4.0600000e+02, 1.3700000e+02, 4.1500000e+02, 2.0600000e+02,
        4.4476023e-01],
       [3.0000000e+00, 7.6000000e+01, 5.6000000e+01, 2.5700000e+02,
        9.8777580e-01],
       [4.4000000e+01, 7.2000000e+01, 1.0600000e+02, 2.5400000e+02,
        9.9625552e-01],
       [3.9400000e+02, 1.3800000e+02, 4.1400000e+02, 2.0400000e+02,
        6.6142339e-01],
       [3.8300000e+02, 1.4400000e+02, 4.0800000e+02, 2.1200000e+02,
        1.1357874e-01],
       [2.7500000e+02, 2.5000000e+01, 3.8000000e+02, 3.4500000e+02,
        9.8586386e-01]], dtype=float32)
 array([], dtype=float32) array([], dtype=float32)
 array([], dtype=float32) array([], dtype=float32)
 array([], dtype=float32) array([], dtype=float32)
 array([], dtype=float32) array([], dtype=float32)
 array([], dtype=float32) array([], dtype=float32)
 array([], dtype=float32) array([], dtype=float32)
 array([], dtype=float32) array([], dtype=float32)
 array([], dtype=float32)
 array([[115.       , 277.       , 295.       , 

Мы видим структуру полей в рамках - это массив классов, где класс это массив экземпляров, где экземпляр это 4 координаты рамки и степень уверенности. Уберем лишнее и дадим имена этим классам. Перебираем их в цикле. Сделаем это по питоновски, через enumerate.

In [7]:
for index, cls in enumerate(coco.classnames):
    if len(boxes[index]) == 0:
        continue
    for obj_box in boxes[index]:
        print("Found class {} in region {} with certainty {:.2f}.".format(cls, obj_box[0:4], obj_box[4]))

Found class person in region [406. 137. 415. 206.] with certainty 0.44.
Found class person in region [  3.  76.  56. 257.] with certainty 0.99.
Found class person in region [ 44.  72. 106. 254.] with certainty 1.00.
Found class person in region [394. 138. 414. 204.] with certainty 0.66.
Found class person in region [383. 144. 408. 212.] with certainty 0.11.
Found class person in region [275.  25. 380. 345.] with certainty 0.99.
Found class dog in region [115. 277. 295. 393.] with certainty 1.00.


Найдено 6 человек и одна собака. Многовато людей. Теперь бы увидеть где эти люди. Для этого проще всего эти рамки нарисовать на кадре и вывести его на экран. Рисовать можно средствами opencv. Сделаем снова перебор классов, как раньше, но с рисованием рамок и имени класса.

In [8]:
for index, cls in enumerate(coco.classnames):
    if len(boxes[index]) == 0:
        continue
    for obj_box in boxes[index]:
        cv2.rectangle(img_to_show,(obj_box[0],obj_box[1]),(obj_box[2],obj_box[3]),(0,255,0),1)
        cv2.putText(img_to_show, "{} {:.2f}".format(cls, obj_box[4]), (obj_box[0],obj_box[1]), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), lineType=cv2.LINE_AA)
        print("Found class {} in region {} with certainty {:.2f}.".format(cls, obj_box[0:4], obj_box[4]))
cv2.imshow("image", img_to_show)
cv2.waitKey(0)
cv2.destroyAllWindows()

Found class person in region [406. 137. 415. 206.] with certainty 0.44.
Found class person in region [  3.  76.  56. 257.] with certainty 0.99.
Found class person in region [ 44.  72. 106. 254.] with certainty 1.00.
Found class person in region [394. 138. 414. 204.] with certainty 0.66.
Found class person in region [383. 144. 408. 212.] with certainty 0.11.
Found class person in region [275.  25. 380. 345.] with certainty 0.99.
Found class dog in region [115. 277. 295. 393.] with certainty 1.00.


В углу картинки найдено 3 человека, которых нет. Давайте объединим всё сделанное в функцию от картинки до рисования рамок. Это предлагается сделать самостоятельно. Она будет получать сессию tensorflow, считанную картинку, порог уверенности в найденном объекте, перечень классов, а выдавать картинку, где эти объекты обведены рамкой, подписан класс и указана степень уверенности.

In [9]:
def yolo(sess, image, threshold, classnames):
    img_to_show = cv2.cvtColor(np.uint8(imge[0]), cv2.COLOR_RGB2BGR)
    image = model.preprocess(image)    
    preds = sess.run(model.preds, {inputs: image})
    boxes = np.array(model.get_boxes(preds, image.shape[1:3]))
    for index, cls in enumerate(classnames):
        if (len(boxes[index]) == 0) :
            continue
        for obj_box in boxes[index]: 
            if  (obj_box[4] > threshold) :                
                cv2.rectangle(img_to_show,(obj_box[0],obj_box[1]),(obj_box[2],obj_box[3]),(0,255,0),1)
                cv2.putText(img_to_show, "{} {:.2f}".format(cls, obj_box[4]), (obj_box[0],obj_box[1]), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), lineType=cv2.LINE_AA)
    return img_to_show

Для вызова этой функции заново считаем изображение (на всякий случай, ведь мы эту переменную уже переприсваивали), создадим сессию, запустим в ней модель и вызовем нашу функцию. Результат нарисуем. Такая схема вызова позволит обрабатывать несколько кадров, не прерывая сессии.

In [ ]:
imge = np.array(nets.utils.load_img('YOLO_test.jpg', target_size=416, crop_size=416))
with tf.Session() as sess:
    sess.run(model.pretrained())
    result = yolo(sess, imge, 0.7, coco.classnames)

cv2.imshow("image", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

Мы умеем распознавать отдельные кадры, подписывать на них нужные данные, ну а теперь распознаем видео! Кадры видео мы будем получать с помощью opencv в виду трёхмерных массивов, так что их надо будет преобразовать к четырёхмерным типа numpy. Также теперь они считываются средствами opencv, так что они сразу в BGR формате и надо его привести к обычному RGB для распознавания.

In [ ]:
with tf.Session() as sess:
    sess.run(model.pretrained())    
    cap = cv2.VideoCapture("yolo_video.webm")
    while(cap.isOpened()): #change the path to your directory or to '0' for webcam
        ret, frame = cap.read()
        if(not ret):
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = crop_square(frame)
        image_to_show = cv2.resize(frame,(416,416))
        imge = np.array(image_to_show).reshape(-1,416,416,3)
        img = yolo(sess, imge, 0.5, coco.classnames)  
        cv2.imshow("image",img)  
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break          

cap.release()
cv2.destroyAllWindows() 

Все кадры распознаются и дополняются данными распознавания. Однако видно, что непрямоугольное изначальное видео сжимается и нарушаются пропорции. Сделай (найди в интернете) функцию откидывания боковых сторон прямоугольника изображения, чтобы оставалась квадратная область. Перезапусти распознавание видео и посмотри стали ли выше вероятности распознавания.

In [10]:
def crop_square(image):
    x_max = image.shape[0]
    y_max = image.shape[1]
    size = min(x_max, y_max)
    crop_image = image[:size, :size]
    return crop_image